## **Занятие 6. Анализ рентгеновского пульсара в режимах аккретор-пропеллер**

В этом практикуме вам предстоит провести самостоятельное исследование *искусственно сгенерированных* данных о пульсациях и поведении рентгеновского источника, для которого наблюдается:
1. Периодичный сигнал с периодом на порядке секунд.
2. Переход между режимами аккретора и пропеллера.

Ваша задача в данной работе:
- Разобраться с чтением данных из fits-файла.
- Разобраться с техническим аспектом Good Time Interval (GTI).
- Нарисовать кривую блеска наблюдения с учетом GTI.
- Сделать вывод о времени перехода между режимами аккретора и пропеллера.
- Вычислить период излучения в каждом из режимов.
- Сделать вывод о физических процессах на объекте.

#### **Подготовительные процедуры**
Перед началом работы **импортируем** нужные библиотеки.\
Убедитесь, что при запуске этой ячейки **не возникает ошибок**.\
При желании, можете посмотреть на реализации функций из `epoch_folding_work`,\
они находятся в отдельном файле с таким же названием.

In [80]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

#### **Достаем данные из fits-файла**.

Для работы с fits-файлами, как и в целом для многих задач, астрофизики создавали\
различные инструменты и приложения, но мы сосредоточимся на одном: модуле astropy,\
а именно его подмодуле `astropy.io.fits`.

Как вы могли заметить, этот модуль уже импортирован выше, осталось понять,\
как его использовать здесь.

Для работы с fits-файлами нам сначала нужно найти сам fits-файл.\
В данном случае нам не придется ходить далеко, он лежит там же,\
где был этот ноутбук, в папке `6_data`. Файл называется `data.fits`.

**_Задание._**\
Найдите этот файл, найдите для него путь, а затем используйте функцию\
`fits.info(путь_к_файлу)`\
чтобы получить информацию о внутренней структуре файла.

In [29]:
# МЕСТО ДЛЯ ВАШЕГО КОДА

In [95]:
fits.info('./6_data/data.fits')

Filename: ./6_data/data.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  DATA          1 BinTableHDU     11   1034771R x 1C   [D]   
  2  GTI           1 BinTableHDU     13   15R x 2C   [D, D]   


Если вы все сделали верно, то должны увидеть список из 3 элементов,\
каждый из которых представляет из себя таблицу.

Cтруктура fits-файла, с которым мы будем работать, такова:
- **0 - PRIMARY**\
Таблица, в которой нет данных, но в заголовке содержится\
вся мета-информация о наблюдении (т.е. полезная информация, которая не\
включает сами данные, например, версию софта обработки, информацию об источнике)
- **1 - DATA**\
Таблица с временем регистрации фотонов нашим детектором.\
В реальных данных тут содержится больше информации: об энергии фотонов,\
о статусе регистрации, технические сведения, необходимые для работы и т.д.
- **2 - GTI**\
Таблица, в которой содержатся интервалы наблюдения, которые пригодны для использования\
в анализе. Обычно такая разбивка связана с различными временными явлениями, например,\
заслонение поля зрения Землей, повышенная Солнечная активность и т.д.

Чтобы получить данные из каждой из таблиц, можно использовать функции\
`fits.getdata(путь_к_файлу)` - получить данные из таблицы.\
`fits.getheader(путь_к_файлу)` - получить заголовое таблицы.

**_Задание 1_.**\
Получите из файла данные о времени регистрации фотонов и промежутках GTI.\
Запишите эти данные в разные переменные.

In [ ]:
# ЗАДАНИЕ 1. МЕСТО ДЛЯ ВАШЕГО КОДА

In [96]:
data = fits.getdata('./6_data/data.fits', 1)
gti = fits.getdata('./6_data/data.fits', 2)

In [97]:
data.dtype

dtype((numpy.record, [('TIME', '>f8')]))

Теперь коротко обсудим, как обращаться к полученным файлам.\
Эти файлы представляют из себя numpy.record, особый тип массивов.

**Для обращения к строчкам таблицы** можно использовать обычную индексацию:\
например, обращение к 11 строчке таблицы имеет вид\
`data[10]`

**Для обращения к столбцам таблицы** необходимо обращаться по имени столбца:\
например, обращение к первому столбцу с именем TIME будет иметь вид\
`data['TIME']`

Чтобы понять, какие имена присвоены столбцам, можно использовать атрибут dtype массива:\
`data.dtype`

_Обратите внимание, что при обращении к строчке таблицы мы получаем tuple, внутри которого_\
_нужно обращаться к элементам строки уже по индексу, а не по имени._

В остальном эти массивы представляют собой обыкновенные numpy-массивы, с которыми можно\
работать так же, как мы обсуждали на предыдущих занятиях.

**_Задание 2._**\
Нарисуйте график, на котором изображено время регистрации первых 100 фотонов в данных\
от положения фотона в таблице.

**_Задание 3._**\
Напишите цикл, который выводит все значения из таблицы GTI в следующем виде:\
`(индекс_строки): START:(число в GTI_START), STOP:(число в GTI_STOP)`\


<p>
<details>
<summary> Дополнительный пункт задания 3 (плохо сформулированный) (кликни, чтобы раскрыть) </summary>

Если вам очень хочется сделать красивый вывод, то попробуйте поработать с f-string:\
При создании строк можно пользоваться таким синтаксисом:\
`f'число, которое я хочу написать: {number}'`\
В результате вы передаете переменную idx напрямую в строку и получите, при `number=12.34`:
> число, которое я хочу написать: 12.34

Интересной особенностью тут является то, что можно изменить формат отображения числа под\
свои нужды, например, автоматически добавлять несколько пробелов:\
`f'число {number:3.2f} -- это то, что я хочу написать'`\
Строка выше будет ожидать числа вида 3 цифры до точки, 2 цифры после точки и автоматически\
постарается привести к этому формату любое доступное число, добавляя лишние пробелы, чтобы\
заполнить пустое пространство.

Это работает по не слишком понятному мне алгоритму, но, если вы хотите привести свои выводы\
к красивому выводу, поэкспериментируйте с f-строчками и покажите, что получится.
</details>
</p>

In [40]:
# ЗАДАНИЕ 2. МЕСТО ДЛЯ ВАШЕГО КОДА

In [39]:
# ЗАДАНИЕ 3. МЕСТО ДЛЯ ВАШЕГО КОДА

## **Работа с GTI.**
Теперь, когда мы посмотрели, как выглядят эти GTI значения, нужно пояснить,\
как с ними работать:\
В таблице GTI есть значения `GTI_START` и `GTI_STOP`, которые обозначают\
начало и конец интервала, данные из которого можно использовать.\
Все данные, которые находятся вне этих интервалов, либо загрязнены каким-то\
внешним явлением, либо в этот момент данных для работы нет вовсе (как в нашем случае).

Таким образом, для работы с данными внутри интервалов, нужно брать данные из таблицы DATA,\
а затем ограничивать интервал для работы границами GTI, чтобы избежать примешивания пустых\
областей без наблюдения в нашу процедуру `epoch_folding`.

**_Задание 4._**\
Для каждого GTI нарисуйте гистограмму числа отсчетов от времени.\
Если у вас 15 GTI, то нарисуйте 15 графиков, скажем, в виде 3x5, и на каждом графике нарисуйте\
гистограммы числа отсчетов в соответствующей GTI.\
Постарайтесь сделать это одним циклом.

Для уменьшения нагрузки на машину при отрисовке гистограммы используйте для `plt.hist` параметр:\
`histtype='stepfilled'`

**_Задание 5._**\
Вычислите среднюю скорость счета для каждого GTI и нарисуйте график зависимости\
скорости счета от времени начала GTI.

**_Часть 1 вопросов об интерпретации результатов._**\
Сделайте вывод о том, что видите. Подумайте и скажите преподавателю, с чем связан вид полученных\
гистограмм и полученный график?

In [ ]:
# ЗАДАНИЕ 4. МЕСТО ДЛЯ ВАШЕГО КОДА

In [ ]:
# ЗАДАНИЕ 5. МЕСТО ДЛЯ ВАШЕГО КОДА

#### **Временной анализ объекта в разных режимах.**

Если вы все сделали правильно, то в прошлом пункте увидели различимое разделение наблюдений на 2 группы,\
в каждой из которых наблюдается разная скорость счета.

Теперь ваша задача исследовать пульсации для каждого из GTI и сделать вывод об изменении периода пульсаций.

**_Часть 2 вопросов об интерпретации результатов._**\
На лекции обсуждалось два режима для рентгеновских пульсара: аккретор и пропеллер.
1. Какой из этих режимов наблюдается в каждом из GTI? Почему?
2. Когда мы можем говорить о наличии периода в данных рентгеновского пульсара? Почему?

**_Задание 6._**\
Найдите период пульсаций в данных тех GTI, для которых считаете это уместным.\
Для этого используйте функцию `search`, импортированную из файла `epoch_folding_work`.\
Она полностью повторяет функцию, которую вы писали на предыдущем занятии.

_Подсказка_:\
Период этого пульсара стоит искать где-то в диапазоне меньше 1 сек.\
Точность, которая вам нужна: не менее 0,0001 сек.

**_Задание 7._**\
Нарисуйте график зависимости периода от времени и найдите производную периода от времени.\
Какой знак имеет эта производная? О каком физическом процессе это свидетельствует? 

In [81]:
from epoch_folding_work import search

In [ ]:
# ЗАДАНИЕ 6. МЕСТО ДЛЯ ВАШЕГО КОДА

In [ ]:
# ЗАДАНИЕ 7. МЕСТО ДЛЯ ВАШЕГО КОДА

#### **Заключительные выводы по заданию.**

В рамках этого задания мы научились работать с данными рентгеновских пульсаров, рисовать их кривые блеска\
в крупном и малом масштабах времени, определили переход между режимами аккретора и пропеллера в рентгеновском\
пульсаре и смогли интерпретировать результаты.\
Конечно, эта работа сильно отличается от деятельности ученых, которые работают с настоящими данными, но она\
представляет собой репрезентативную модель того, чем вам потенциально предстоит заниматься в астрофизике\
высоких энергий.